In [1]:
from obsutils.utils import SkyportalClient, get_target_list, get_run_info
from obsutils.data import standards_df
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroplan import Observer
from astroplan import FixedTarget
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroplan.plots import plot_airmass
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
run = 2080

In [3]:
run_info = get_run_info(run)

In [4]:
night = run_info["calendar_date"].replace("-", "")
night

'20251214'

In [5]:
df = get_target_list(run)
df.sort_values(by=["priority"], ascending=False)[["id", "tns_name", "priority", "comment"]]

,id,tns_name,priority,comment
3,ZTF25abxvhmk,TDE 2025abcr,4,
4,ZTF25acfcmnb,AT 2025afje,4,"Blue/featureless, little host"
9,ZTF25achcqbz,AT 2025agqp,4,Super blue
0,ZTF24abkrttp,AT 2024aiif,3,"brightish, nuclear, possibly Ia-ish"
2,ZTF18abquopi,AT 2025agsf,3,Classify because it's close
5,ZTF25acguwni,AT 2025agtt,3,"seems reasonable, difficult for Lick"
7,ZTF25acgqggy,SN 2025agkk,3,"Bright, not bad"
8,ZTF25acgqhlx,AT 2025aebu,3,"Flat, blue"
1,ZTF25acfyeke,AT 2025afvr,2,Great candidate but have a spec already
6,ZTF25acgqfpt,AT 2025agqe,2,"fast, blue, faint host"


In [6]:
targets = []
tcslist = ['#title=true ra=true dec=true epoch=false muRA=false muDec=false ' +
           'magnitude=false dRA=false dDec=false rotatorPA=true ' +
           'rotatorFrame=true comment=false']

# for i in obslist.index:
for _, row in df.iterrows():
    coords = SkyCoord(ra=row["ra"], dec=row["dec"],
                      unit=(u.deg, u.deg))
    targ = FixedTarget(coord=coords, name=row["id"])
    targets.append(targ)
    fmt_coord = coords.to_string('hmsdms', sep=':', pad=True, precision=2)
    tcslist.append(row["id"] + ' ' + fmt_coord + ' 180 Fixed')

for i, row in standards_df.iterrows():
    name = row["name"]
    coords = SkyCoord(ra=row["ra"], dec=row["dec"], unit=(u.hourangle, u.deg))
    targ = FixedTarget(coord=coords, name=row["name"])
    targets.append(targ)
    fmt_coord = coords.to_string('hmsdms', sep=':', pad=True, precision=2)
    tcslist.append(name + ' ' + fmt_coord + ' 180 Fixed')

tcslist

['#title=true ra=true dec=true epoch=false muRA=false muDec=false magnitude=false dRA=false dDec=false rotatorPA=true rotatorFrame=true comment=false',
 'ZTF24abkrttp 00:09:54.42 +03:41:42.67 180 Fixed',
 'ZTF25acfyeke 01:07:36.29 +24:22:39.83 180 Fixed',
 'ZTF18abquopi 01:35:25.62 -05:40:43.87 180 Fixed',
 'ZTF25abxvhmk 01:46:55.39 -15:22:15.60 180 Fixed',
 'ZTF25acfcmnb 02:48:09.88 +37:45:17.37 180 Fixed',
 'ZTF25acguwni 02:56:36.59 +77:44:25.78 180 Fixed',
 'ZTF25acgqfpt 07:56:51.78 +53:19:26.99 180 Fixed',
 'ZTF25acgqggy 07:57:30.59 +39:35:26.26 180 Fixed',
 'ZTF25acgqhlx 09:00:41.57 +33:26:28.61 180 Fixed',
 'ZTF25achcqbz 10:54:19.88 -16:46:00.73 180 Fixed',
 'ZTF25acgmrbq 11:59:33.30 +22:41:34.96 180 Fixed',
 'G191-B2B 05:05:30.60 +52:49:56.00 180 Fixed',
 'Feige34 10:39:36.71 +43:06:10.10 180 Fixed',
 'Feige67 12:41:51.83 +17:31:20.50 180 Fixed',
 'Feige110 23:19:58.39 -05:09:55.80 180 Fixed',
 'BD+28d4211 21:51:11.07 +28:51:51.80 180 Fixed',
 'BD332642 15:51:59.89 +32:56:54.30 

In [7]:
load_dotenv()
base_output_dir = Path(os.getenv("OUTPUT_DIR", str(Path.home()) + "/obsutils"))

print(base_output_dir)

output_dir = base_output_dir / night
output_dir.mkdir(parents=True, exist_ok=True)


output_path = output_dir / (night + '.tls')

print(f"Saving to {output_path}")
np.savetxt(output_path, tcslist, fmt='%s')

/Users/rdstein/Data/obsutils
Saving to /Users/rdstein/Data/obsutils/20251214/20251214.tls


In [8]:
# # Set observer at DCT
# dct = Observer.at_site('Lowell')

# ############ CHANGE THESE LINES #############
# #############################################
# start = Time('2025-02-28 01:22', scale='utc')
# end = Time('2025-02-28 07:37', scale='utc')
# #############################################

# delta_t = end-start
# obstime = start + delta_t*np.linspace(0, 1, 75)

# # Colors used to tell targets apart - may need to add more
# color_idx = np.array(['r', 'b', 'g', 'teal', 'orange', 'purple', 'k',
#                       'lime', 'cornflowerblue', 'dimgrey', 'cyan',
#                       'blueviolet', 'deeppink', 'y', 'olive', 'yellow',
#                       'deepskyblue', 'darksalmon', 'yellowgreen', 'aliceblue',
#                       'steelblue', 'firebrick', 'violet', 'maroon',
#                       'darkkhaki', 'mediumslateblue', 'plum', 'crimson',
#                       'sandybrown', 'antiquewhite', 'indigo'])

# # targets = np.array(targets)
# # targets = np.delete(targets, [17, 21, 20, 19, 18])

# plt.figure(figsize=(10, 5))
# # Plot airmass
# for i in range(len(targets)):
#     shade = False
#     if i == 0:  # Used to avoid issues with doing multiple shades?
#         shade = True
#     bonk = plot_airmass(targets[i], dct, obstime, brightness_shading=shade,
#                  style_kwargs=dict(color=color_idx[i]),
#                  use_local_tz=True, min_airmass=1.0, max_airmass=2.0)

# ticks = bonk.get_xticklabels()
# newlabels = []
# for t in ticks:
#     time = int(t.get_text()[:2])
#     if time > 7:
#         time -= 7
#     elif time == 7:
#         time = 12
#     elif time < 7:
#         time = 12 - (7-time)
#     newlabels.append(str(time)+":00")
    
# bonk.set_xticklabels(newlabels)
# plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=3)
# #plt.grid()

# ########### CHANGE THIS LINE ######################
# ###################################################
# plt.xlabel('MST')
# ###################################################

# plt.savefig('./'+obsdate+'/LDT-airmass.pdf', dpi=400, bbox_inches='tight')
# plt.show()

In [9]:
# # Plot sky
# for i in range(len(targets)):
#     plot_sky(targets[i], dct, obstime, north_to_east_ccw=False,
#              style_kwargs=dict(color=color_idx[i]))

# plt.legend(loc=(1.14, 0))
# # plt.savefig('DCT-sky.png', dpi=300, bbox_inches='tight')
# plt.show()

# # Plot altitude
# for i in range(len(targets)):
#     shade = False
#     if i == 0:
#         shade = True
#     plot_altitude(targets[i], dct, obstime, brightness_shading=shade,
#                   style_kwargs=dict(color=color_idx[i]))

# plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.8), ncol=3)

# ########### CHANGE THIS LINE ######################
# ###################################################
# plt.xlabel('Time from 2019-08-18 [UTC] (MST+7:00)')
# ###################################################

# # plt.savefig('DCT-alt.png', dpi=300, bbox_inches='tight')
# plt.show()